# Linear regression from scratch

In this notebook, I want to show you some "from scratch" linear regression examples. The purpose is to better understand a couple of ways we can train a linear regression model (said differently: how we can learn the weights for a linear regression model). In a later notebook, I'll show you how to use Pandas and Scikit-learn.

I'm importing housing price data from [housing-all.csv](https://u.pcloud.link/publink/show?code=XZg96Y5Z8xq8LWJ7URSeFTOzymFjGkN5OSaV).

In [81]:
import numpy as np
import numpy.linalg as linalg
import math

xs = []
ys = []
with open('../data/housing-all.csv', 'r') as file:
    columns = ['intercept'] + file.readline().strip().split(',')
    for line in file:
        row = [float(value) for value in line.strip().split(',')]
        if len(row) > 0:
            xs.append([1] + row[:-1]) # All the features go into a new row in xs, plus 1 to serve as the intercept.
            ys.append(row[-1]) # The dependent variable (last column) goes into ys.

# Split data into training and dev.
trainDevThreshold = int(0.8*len(xs))
trainXs = np.array(xs[:trainDevThreshold]) # 80% goes to training, 20% to dev.
trainYs = np.array(ys[:trainDevThreshold])

devXs = np.array(xs[trainDevThreshold:])
devYs = np.array(ys[trainDevThreshold:])

def minMaxScale(values, min, max):
    if min == max:
        return
    for i in range(len(values)):
        values[i] = (values[i] - min)/(max - min)

# Scale features using trianing data. We'll just use min-max scaling.
minMaxes = [( min(trainXs[:,col]), max(trainXs[:,col])) for col in range(len(trainXs[0]))]
for col,minMax in enumerate(minMaxes):
    colMin, colMax = minMax
    minMaxScale(trainXs[:,col], colMin, colMax)
    minMaxScale(devXs[:,col], colMin, colMax)

trainXs, trainYs

(array([[1.        , 0.68117409, 0.09458023, ..., 0.05456992, 0.14109521,
         0.24584488],
        [1.        , 0.60526316, 0.13496281, ..., 0.02281454, 0.05525407,
         0.31911284],
        [1.        , 0.65587045, 0.16259299, ..., 0.06185711, 0.12613057,
         0.22874167],
        ...,
        [1.        , 0.31882591, 0.65037194, ..., 0.02043219, 0.03946719,
         0.34276079],
        [1.        , 0.63259109, 0.16259299, ..., 0.10686959, 0.15293537,
         0.13865326],
        [1.        , 0.60728745, 0.17321998, ..., 0.01014602, 0.02795593,
         0.10610888]], shape=(16346, 9)),
 array([240600., 458300., 258700., ..., 204800., 185000., 225000.],
       shape=(16346,)))

In [82]:
trainXs[:,0]

array([1., 1., 1., ..., 1., 1., 1.], shape=(16346,))

## Ordinary Least Squares

This is one of several least squares approaches. It is closed form, meaning we can use a formula of basic operations to manipulate the data to arrive at the final weights that minimize error, as opposed to an interative approach where we continually refine weights until we reach an error we're comfortable with (we'll look at an example of that next!).

The formula is:

$$\beta = \left(X^TX\right)^{-1} X^Ty$$

where $X$ is the feature *matrix* (a 2D array) where rows are observations and columns are features, and $y$ the vector (a 1D array) of true values for each of the observations. The linear algebra operations we see left to right are *transpose* ($X^T$), which is necessary for the next operation, *matrix multiplication*, where we are multiplying $X$ by itself. Next, we take the *inverse* ($X^{-1}$) of the result (this finds another matrix that, when multiplied with $X$ yields an identity matrix, which is the equivalent of the number "1" in matrix land). We multiply that result with the transpose of $X$ again, then finally multiply the resulting matrix with the vector $y$.

In [83]:
# The "@" are used by Numpy for matrix/vector multiplication.
weights = (linalg.inv(trainXs.T @ trainXs) @ trainXs.T) @ trainYs

weights

array([  369914.73252504,  -426310.32627907,  -404218.66258238,
          57873.48435623,  -331098.74881244,   779856.23326378,
       -1312274.72168028,   223637.35618937,   583329.64477493])

In [84]:
for feature,weight in zip(columns, weights):
    print(f'{feature:<20}: {weight:>15.4f}')

intercept           :     369914.7325
longitude           :    -426310.3263
latitude            :    -404218.6626
housing_median_age  :      57873.4844
total_rooms         :    -331098.7488
total_bedrooms      :     779856.2333
population          :   -1312274.7217
households          :     223637.3562
median_income       :     583329.6448


In [76]:
# Evaluate -- how well do we predict the dev set?
errors = devYs - devXs @ weights
print('Root mean squared error:', 
    math.sqrt(sum([error**2 for error in errors])/len(devYs)))

Root mean squared error: 68881.30146276834


In [85]:
devXs[-1,]

array([1.        , 0.21963563, 0.60467588, 0.39215686, 0.0628974 ,
       0.05276226, 0.0226744 , 0.05048512, 0.4661315 ])

In [86]:
1*369914.7325 + 0.21963563*-426310.3263 + 0.60467588*-404218.6626 + 0.39215686*57873.4844 + 0.0628974*-331098.7488 + \
0.05276226*779856.2333 + 0.0226744*-1312274.7217 + 0.05048512*223637.3562 + 0.4661315*583329.6448


328321.36984791735

## Stocastic gradient decent

This is an iterative method that performs a number of *epochs*. In one epoch, each of the observations in the training set are iterated over. On each iteration, the weights are adjusted based on the difference between the prediction using the most recent weights and the true dependent varaible (y) for the given observation ($h$ in the example below, which let's imagine is short for *hypothesis*), scaled by both $alpha$ (the learning rate) and the feature value that corresponds with the weight being updated.

In the example below, I've set the learning rate to 0.01 and the number of epochs to 50; these can both be tweaked. We might also shuffle the observations since that might have an effect on how much weights are adjusted.

In [91]:
# Initialize weights + intercept to 1.
weights = np.array([1]*(len(columns)-1)) # -1 because y is included in the column names.
# This is our learning rate -- how much to adjust weights by relative to error (smaller values = slower, but more accurate)
alpha = 0.001

print('Epoch ', end='')
for epoch in range(50):
    print(f'{epoch},', end='')
    for row, y in zip(trainXs, trainYs):
        h = sum([x*weight for (x,weight) in zip(row, weights)])

        # Update each of the weights.
        for i in range(len(weights)):
            weights[i] = weights[i] - alpha*(h - y)*row[i]

weights

Epoch 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,

array([ 171984, -176659, -179908,   72044,      14,   59933,       7,
         10409,  571664])

In [92]:
for feature,weight in zip(columns, weights):
    print(f'{feature:<20}: {weight:>15.4f}')

intercept           :     171984.0000
longitude           :    -176659.0000
latitude            :    -179908.0000
housing_median_age  :      72044.0000
total_rooms         :         14.0000
total_bedrooms      :      59933.0000
population          :          7.0000
households          :      10409.0000
median_income       :     571664.0000


In [93]:
import math

# We could calculated RMSE the same as above, but let's try it down here without matrix multiplication.
errors = []
for devX, devY in zip(devXs, devYs):
    # Calculates the error between each predicted dev Y and the actual dev Y
    errors.append(sum([x*weight for (x,weight) in zip(devX,weights)]) - devY)

sumSquaredErrors = sum([error**2 for error in errors])

print('Root mean squared error:', 
    math.sqrt(sumSquaredErrors/len(devXs)))

Root mean squared error: 74858.75824406656
